## Problem 4: What is the closest shopping center from your home / work? (5 points)

In the last problem you should find out the closest shopping center from two different locations a) your home and b) work place. 

**Steps**:

 - Create a txt-file called `activity_locations.txt` (use same formatting as in Problem 1) with two columns:
    - `id`: unique id for each row
    - `addr`:  address of your work and home (or any two addresses in the Helsinki Region.)
    
Save the text file into this repository.
    
 - Read those addresses using pandas and geocode the addresses.
 - Find out the nearest shopping center to these points using Shapely `nearest_points`.
 - Print out the name of the shopping center that is nearest to a) home and b) work. For example: `Shopping center closest to home: REDI`.

In [1]:
#import 
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
# read in input files
addresses = pd.read_csv('addresses.txt', sep=';')
shopping_center = pd.read_csv('shopping_centers.txt', sep=';')

In [3]:
# Geocode activity locations
#import geocoing tool
from geopandas.tools import geocode

addresses_geo = geocode(addresses['addr'], provider='nominatim', user_agent='autogis_HA', timeout=5)
shopping_center_geo = geocode(shopping_center['addr'], provider='nominatim', user_agent='autogis_HA', timeout=5)

In [4]:
# Check coordinate reference system information (re-project/define if needed)
print(addresses_geo.crs)
print(shopping_center_geo.crs)

EPSG:4326
EPSG:4326


In [5]:
addresses_geo = addresses_geo.to_crs(epsg=3879)
shopping_center_geo = shopping_center_geo.to_crs(epsg=3879)

print(addresses_geo.crs)
print(shopping_center_geo.crs)

EPSG:3879
EPSG:3879


In [6]:
shopping_center_geo.head(10)

,geometry,address
0,POINT (25504598.602 6677662.109),"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va..."
1,POINT (25496573.542 6672878.360),"Salaattiasema, 14-20, Mannerheimintie, Keskust..."
2,POINT (25485470.435 6672070.967),"Iso Omena, 11, Piispansilta, Matinkylä, Suur-M..."
3,POINT (25489428.008 6678393.041),"William K, 3-9, Leppävaarankatu, Säteri, Etelä..."
4,POINT (25497943.932 6686656.982),"Stockmann, 3, Vantaanportinkatu, Vantaanportti..."
5,POINT (25498869.737 6674838.689),"Oluthuone Haavi, 5, Hermannin rantatie, Verkko..."
6,POINT (25496166.681 6676094.626),"Pasilansilta, Keski-Pasila, Pasila, Keskinen s..."


In [9]:
shopping_center_join =  shopping_center_geo.join(shopping_center)
shopping_center_join = shopping_center_join[['geometry', 'id', 'name']]
shopping_center_join.head()

,geometry,id,name
0,POINT (25504598.602 6677662.109),0,Itis
1,POINT (25496573.542 6672878.360),1,Forum
2,POINT (25485470.435 6672070.967),2,Iso-Omena
3,POINT (25489428.008 6678393.041),3,Sello
4,POINT (25497943.932 6686656.982),4,Jumbo


In [11]:
addresses_geo.head()

,geometry,address
0,POINT (25495311.608 6672258.695),"Ruoholahti, 14, Itämerenkatu, Ruoholahti, Läns..."
1,POINT (25496206.216 6672909.016),"Kamppi, 1, Kampinkuja, Kamppi, Eteläinen suurp..."
2,POINT (25496762.723 6673010.538),"Bangkok9, 8, Kaivokatu, Keskusta, Kluuvi, Etel..."
3,POINT (25498815.142 6675243.635),"Hermannin rantatie, Verkkosaari, Kalasatama, S..."
4,POINT (25495641.151 6671525.076),"9, Tyynenmerenkatu, Jätkäsaari, Länsisatama, E..."


In [13]:
# Find the closest shopping center for each activity location (closest shopping center from
# and closest shopping center from work) and print out the results:
from shapely.ops import nearest_points

#create function for nearest values
def get_nearest_center(row, other_gdf, point_column='geometry', value_column='geometry'):

    #create union(multipoints)
    other_points = other_gdf['geometry'].unary_union
    
    #find nearest points
    nearest_geoms = nearest_points(row[point_column], other_points)

    #get correspomding values
    nearest_data = other_gdf.loc[other_gdf['geometry'] == nearest_geoms[1]]

    nearest_value = nearest_data[value_column].values[0]

    return nearest_value

In [15]:
#apply function
addresses_geo['nearest_center'] = addresses_geo.apply(get_nearest_center, other_gdf=shopping_center_join, value_column='name', axis=1)

In [17]:
addresses_geo

,geometry,address,nearest_center
0,POINT (25495311.608 6672258.695),"Ruoholahti, 14, Itämerenkatu, Ruoholahti, Läns...",Forum
1,POINT (25496206.216 6672909.016),"Kamppi, 1, Kampinkuja, Kamppi, Eteläinen suurp...",Forum
2,POINT (25496762.723 6673010.538),"Bangkok9, 8, Kaivokatu, Keskusta, Kluuvi, Etel...",Forum
3,POINT (25498815.142 6675243.635),"Hermannin rantatie, Verkkosaari, Kalasatama, S...",REDI
4,POINT (25495641.151 6671525.076),"9, Tyynenmerenkatu, Jätkäsaari, Länsisatama, E...",Forum
5,POINT (25504528.607 6680282.118),"18, Kontulantie, Kontula, Mellunkylä, Itäinen ...",Itis
6,POINT (25505903.642 6678540.297),"Itäväylä, Vartioharju, Vartiokylä, Itäinen suu...",Itis
7,POINT (25501631.384 6685110.943),"Tapulikaupungintie, Tapulikaupunki, Suutarila,...",Jumbo
8,POINT (25501586.789 6683452.706),"Sompionpolku, Fallkullan kiila, Tapanila, Tapa...",Jumbo
9,POINT (25492851.783 6678869.234),"Sodexo, 5, Atomitie, Strömberg, Pitäjänmäen te...",Sello


Can you think of other application cases for the nearest neighbour analysis?

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()